In [1]:
import numpy as np
from scipy.sparse import csr_matrix as sparse_mat
import pickle
import sys

In [2]:
def non_zero(sparse_matrix):
    return np.transpose(sparse_matrix.nonzero())[:1000, :]

In [3]:
def mean_nonzero(sparse_matrix):
    points = non_zero(sparse_matrix)
    sum = 0
    for point in points:
        sum += c[point[0], point[1]]
    return sum / len(points)

In [4]:
ratings = pickle.load(open('ratings_matrix.p', 'rb'))

In [5]:
ratings.shape

(552339, 77445)

In [6]:
def train(R, k=2, eta=1e-2, lamb=1e-2, threshold=0.15, iterations=2500):
    m, n = R.shape
    U = np.random.uniform(-1, 1, (m, k))
    B = np.random.uniform(-1, 1, (n, k))
    i = 0
    prev = 0
    curr = sys.maxint
    while i < iterations and abs(curr - prev) > threshold:
        prev = curr
        cost = 0
        for idx in non_zero(R):
            epsilon = R[idx[0], idx[1]] - np.dot(U[idx[0]], B[idx[1]].T)
            U[idx[0], :] = U[idx[0], :] + eta * (epsilon * B[idx[1]] - lamb * U[idx[0], :])
            B[idx[1], :] = B[idx[1], :] + eta * (epsilon * U[idx[0]] - lamb * B[idx[1], :])
            cost += (R[idx[0], idx[1]] - np.dot(U[idx[0]], B[idx[1]].T )) ** 2 + \
                                    lamb * (np.linalg.norm(U[idx[0]])**2 + np.linalg.norm(B[idx[1]])**2)

        cost = cost / len(non_zero(R))
        curr = cost
        i += 1
        
    print "Total Iterations: ", i
    return U, B

In [44]:
user_map = pickle.load(open('users_map.p', 'rb'))
business_map = pickle.load(open('business_map.p', 'rb'))
reviews = pickle.load(open('reviews.p', 'rb'))

In [7]:
baselines = pickle.load(open('baselines.p', 'rb'))

In [9]:
alpha = baselines['alpha'][0]
beta_users = baselines['beta_users']
beta_business = baselines['beta_business']

In [15]:
# removing baselines from ratings data
for idx in non_zero(ratings):
    ratings[idx[0], idx[1]] -= (alpha + beta_users[idx[0]] + beta_business[idx[1]])

IndexError: tuple index out of range

In [14]:
beta_business[1]

-0.2560921134291414

In [ ]:
U, B = train(ratings, k=10)